In [ ]:
import spotipy
import spotipy.util as util
import pandas as pd
import os
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime
from datetime import timedelta, date
import glob
from os import listdir
import numpy as np
import pymongo
import csv
import json
import sys, getopt, pprint
from pymongo import MongoClient
import types
import itertools
from itertools import zip_longest

In [ ]:
def current_user_recently_played(self, limit=50):
    return self._get('me/player/recently-played', limit=limit)

In [ ]:
scope="user-read-recently-played user-read-private user-top-read user-read-currently-playing"

In [ ]:
if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print ("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:
token = util.prompt_for_user_token(username,scope,
                           client_id='e67d7b1bc1ee460bbef7484ee3f601ef',
                           client_secret='ba0087b5f4ab4fb8ab88be1850854538',
                           redirect_uri='https://google.com')

In [ ]:
spotify = spotipy.Spotify(auth=token)
spotify.current_user_recently_played = types.MethodType(current_user_recently_played, spotify)

In [ ]:
dave = spotify.current_user_recently_played(limit=50)
out_file = open("dave.json","w")
out_file.write(json.dumps(dave, sort_keys=True, indent=2))
out_file.close()

#print(json.dumps(dave, sort_keys=True, indent=2))

In [ ]:
Tracks = []

for i in range(50):
   Tracks.append(dave['items'][i]['track']['name'])

TrackID = []

for i in range(50):
   TrackID.append(dave['items'][i]['track']['id'])

Artists = []

for i in range(50):
   Artists.append(dave['items'][i]['track']['artists'][0]['name'])

Albums = []


for i in range(50):
   Albums.append(dave['items'][i]['track']['album']['name'])

for (a, b, c, d) in itertools.zip_longest(TrackID, Tracks, Artists, Albums): 
    print (a, b, c,d )

In [ ]:
dave_df = pd.DataFrame(np.column_stack([TrackID, Tracks, Artists, Albums]), 
                               columns=['Track ID','Tracks', 'Artists', 'Albums'])

dave_df.head(1)

In [ ]:
features = []
for i in range(0,len(TrackID),50):
    audio_features = spotify.audio_features(TrackID[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 1

features

In [ ]:
audiofeat_df = pd.DataFrame(features)
audiofeat_df.head()

In [ ]:
audiofeat_df.columns

In [ ]:
audiofeat_df.drop(audiofeat_df.loc[:,'uri':'target'], inplace = True, axis = 1)

In [ ]:
del audiofeat_df['type']

In [ ]:
audiofeat_df.head(1)

In [ ]:
DaveDF = dave_df.join(audiofeat_df)

In [ ]:
del DaveDF['id']

In [39]:
DaveDF.to_csv('dfs/DaveDF.csv',index=False)

DaveDF

,Track ID,Tracks,Artists,Albums,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,340FrzH0Ji2keVAlX2FJZ5,At Your Funeral,Saves The Day,Stay What You Are,0.257,0.936,2,-6.131,1,0.3570,0.002430,0.000000,0.1400,0.376,197.929
1,7CKLZEwboCy2juXMLsAHFt,Whoa Oh! (Me vs Everyone),Forever The Sickest Kids,Underdog Alma Mater,0.529,0.923,2,-4.166,1,0.0764,0.003930,0.000000,0.3330,0.640,143.034
2,4HAWq1kBDMLSNq5UHWD7rH,Somewhere On Fullerton,Allister,Last Stop Suburbia,0.540,0.912,3,-4.662,1,0.0855,0.003150,0.000000,0.1240,0.828,96.958
3,1rUHLQEmtefobuukC1SVDf,Bouncing Off The Walls,Sugarcult,Start Static,0.477,0.917,6,-4.692,1,0.0547,0.002950,0.000000,0.2680,0.710,150.974
4,6fx1fKIs5e0pBL5FpdC7RB,MakeDamnSure,Taking Back Sunday,Louder Now (Deluxe Edition),0.553,0.837,6,-5.608,1,0.0452,0.009410,0.000000,0.3720,0.432,140.495
5,0iWZd4aBNd3vNUF6crMiOp,Hold My Hand,New Found Glory,Coming Home,0.478,0.963,11,-3.299,1,0.0643,0.002980,0.000000,0.1650,0.899,164.641
6,6vFD3c1WI1zuKoyk22dMw3,Little Things,Good Charlotte,Good Charlotte,0.686,0.840,1,-5.142,1,0.1590,0.060100,0.000005,0.1640,0.704,105.000
7,3B9W8NK6FiaNZCZFIzLdwo,In This Diary,The Ataris,"So Long, Astoria",0.566,0.832,8,-4.644,1,0.0603,0.005050,0.000000,0.1270,0.860,175.028
8,63k8KXOgawK2cNtvm2ZW8y,Can't Be Saved,Senses Fail,Still Searching,0.331,0.976,2,-3.242,1,0.1130,0.000009,0.000117,0.3190,0.426,170.059
9,6uzuW7L1t1UhnzbfXwtMtQ,The Curse of Curves,Cute Is What We Aim For,The Same Old Blood Rush With A New Touch,0.534,0.792,5,-4.512,1,0.0677,0.390000,0.000000,0.1910,0.477,145.881


In [ ]:
DaveDF2 = DaveDF[['Track ID','danceability','energy',
                  'speechiness', 'acousticness',
                  'instrumentalness', 'liveness', 'valence']]

DaveDF2.to_csv('dfs/DaveDF2.csv',index=False)

DaveDF2

In [38]:
DaveDF.columns

Index(['Track ID', 'Tracks', 'Artists', 'Albums', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')